# Read data example

In [1]:
import os
import typing
from tqdm import tqdm
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json

In [2]:
ROOT_DATA_FOLDER = r"/kaggle/input/self-drive-yandex-cup/YandexCup24SelfDrivingCars"

TRAIN_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, r"YaCupTrain")

TEST_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, r"YaCupTest")
label_columns = ['x', 'y', 'yaw']


In [3]:
# Load all ids of a dataset

def read_testcase_ids(dataset_path: str):
    ids = [int(case_id) for case_id in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, case_id))]
    return ids

ids = read_testcase_ids(TRAIN_DATASET_PATH)
len(ids)

train_ids = np.random.choice(ids, size=round(0.70*len(ids)), replace=False, p=None)
test_ids = [el for el in ids if el not in train_ids]

In [4]:
class DataFilePaths:
    def __init__(self, testcase_path: str):
        self.testcase_path = testcase_path

    def localization(self):
        return os.path.join(self.testcase_path, 'localization.csv')

    def control(self):
        return os.path.join(self.testcase_path, 'control.csv')

    def metadata(self):
        return os.path.join(self.testcase_path, 'metadata.json')

    # exists only for test_dataset
    def requested_stamps(self):
        return os.path.join(self.testcase_path, 'requested_stamps.csv')

In [5]:


def read_localization(localization_path: str):
    return pd.read_csv(localization_path)

def read_control(control_path):
    return pd.read_csv(control_path)

def read_metadata(metadata_path: str):
    with open(metadata_path, 'r') as f:
        data = json.load(f)
    return data

def read_requested_stamps(requested_stamps_path: str):
    return pd.read_csv(requested_stamps_path)

def read_testcase(dataset_path: str, testcase_id: str, is_test: bool = False):
    testcase_path = os.path.join(dataset_path, str(testcase_id))
    data_file_paths = DataFilePaths(testcase_path)

    testcase_data = {}
    testcase_data['localization'] = read_localization(data_file_paths.localization())
    testcase_data['control'] = read_control(data_file_paths.control())
    testcase_data['metadata'] = read_metadata(data_file_paths.metadata())
    if is_test:
        testcase_data['requested_stamps'] = read_requested_stamps(data_file_paths.requested_stamps())

    return testcase_data

In [6]:
def read_testcases(dataset_path: str, is_test: bool = False, testcase_ids: typing.Iterable[int] = None):
    result = {}
    if testcase_ids is None:
        testcase_ids = read_testcase_ids(dataset_path)

    for testcase_id in tqdm(testcase_ids):
        testcase = read_testcase(dataset_path, testcase_id, is_test=is_test)
        result[testcase_id] = testcase
    return result

In [7]:
TRAIN_DATASET_PATH

'/kaggle/input/self-drive-yandex-cup/YandexCup24SelfDrivingCars/YaCupTrain'

In [8]:
# may take some time

train_dataset = read_testcases(TRAIN_DATASET_PATH)
len(train_dataset)

100%|██████████| 42000/42000 [19:57<00:00, 35.07it/s]


42000

In [51]:

train = {k: v for k, v in train_dataset.items() if k in train_ids}
test = {k: v for k, v in train_dataset.items() if k in test_ids}


In [46]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [47]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels


WindowGenerator.split_window = split_window

In [53]:
w2 = WindowGenerator(train_df=train[3]['localization'], val_df=test[2]['localization'], label_columns=['x','y','yaw'], input_width=5, label_width=5, shift=1)

In [54]:
example_window = tf.stack([np.array(train[3]['localization'][:w2.total_window_size]),
                           np.array(train[3]['localization'][100:100+w2.total_window_size]),
                           np.array(train[3]['localization'][200:200+w2.total_window_size])
                           ])



In [56]:
example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')


All shapes are: (batch, time, features)
Window shape: (3, 6, 7)
Inputs shape: (3, 5, 7)
Labels shape: (3, 5, 3)


In [57]:
def make_dataset(self, data):
    data = np.array(el, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

In [24]:
len(train)

29400

In [58]:
@property
def train(self):
    for i in train_ids:
        train_df = train[i]['localization']
        return self.make_dataset(self.train_df)



@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val

In [64]:
!install paramiko

install: missing destination file operand after 'paramiko'
Try 'install --help' for more information.


In [61]:
wide_window = WindowGenerator( train_df=train[3]['localization'], val_df=test[2]['localization'], label_columns=['x','y','yaw'],
    input_width=24, label_width=24, shift=1,)

wide_window

TypeError: 'property' object is not subscriptable

In [67]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [65]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [68]:
history = compile_and_fit(lstm_model, w2)

AttributeError: 'WindowGenerator' object has no attribute 'train'

In [ ]:
train_dataset[5]['control']['stamp_ns'] = train_dataset[5]['control']['stamp_ns']/10e9
train_dataset[5]['localization']['stamp_ns'] = train_dataset[5]['localization']['stamp_ns']/10e9

In [ ]:
train_df = pd.DataFrame()
for el in train_dataset:
    

In [ ]:
train_dataset[25]['control']

,stamp_ns,acceleration_level,steering
0,2998480760,-4500,-7.419072
1,3038053702,-4500,-7.419072
2,3078267726,-4500,-7.419072
3,3118356686,-4500,-7.419072
4,3158379017,-4500,-7.419072
...,...,...,...
1495,62798345848,5969,-5.625576
1496,62838498438,5911,-5.688811
1497,62878325068,5816,-5.688811
1498,62918339890,5674,-5.762313


In [ ]:
train_dataset[25]['control'].drop(columns='stamp_ns')

,acceleration_level,steering
0,-4500,-7.419072
1,-4500,-7.419072
2,-4500,-7.419072
3,-4500,-7.419072
4,-4500,-7.419072
...,...,...
1495,5969,-5.625576
1496,5911,-5.688811
1497,5816,-5.688811
1498,5674,-5.762313


In [ ]:
pd.concat([df1.set_index('A'),df2.set_index('A')], axis=1, join='inner').reset_index()


In [ ]:
a = train_dataset[25]['localization'][74:1574]

In [ ]:
train_dataset[25]['control'].drop(columns='stamp_ns')

,acceleration_level,steering
0,-4500,-7.419072
1,-4500,-7.419072
2,-4500,-7.419072
3,-4500,-7.419072
4,-4500,-7.419072
...,...,...
1495,5969,-5.625576
1496,5911,-5.688811
1497,5816,-5.688811
1498,5674,-5.762313


In [ ]:
pd.concat([train_dataset[25]['localization'][74:1574].reset_index(),train_dataset[25]['control'].drop(columns='stamp_ns').reset_index()], axis=1).reset_index().drop(columns=['index','level_0'])

,stamp_ns,x,y,z,roll,pitch,yaw,acceleration_level,steering
0,2960529547,148.959645,-297.067037,-21.485054,-0.008272,-0.007133,2.674813,-4500,-7.419072
1,3000472113,148.959645,-297.067038,-21.485054,-0.008274,-0.007132,2.674808,-4500,-7.419072
2,3039900228,148.959645,-297.067038,-21.485054,-0.008273,-0.007132,2.674806,-4500,-7.419072
3,3080404108,148.959508,-297.066878,-21.484694,-0.008322,-0.007116,2.674797,-4500,-7.419072
4,3119984778,148.959507,-297.066878,-21.484694,-0.008318,-0.007116,2.674795,-4500,-7.419072
...,...,...,...,...,...,...,...,...,...
1495,62813930520,11.228409,-297.296181,-19.815981,0.004062,-0.013882,2.460280,5969,-5.625576
1496,62853584035,10.994059,-297.106072,-19.800928,0.007370,-0.018068,2.460444,5911,-5.688811
1497,62893488384,10.761062,-296.917137,-19.795043,0.010270,-0.020149,2.460652,5816,-5.688811
1498,62934115579,10.524072,-296.724955,-19.788326,0.016922,-0.022580,2.460336,5674,-5.762313


In [ ]:
test_dataset = read_testcases(TEST_DATASET_PATH, is_test=True)
len(test_dataset)

100%|██████████| 8000/8000 [04:30<00:00, 29.52it/s]


8000

In [ ]:
loc stams x y z roll pitch yaw 0--> LSTM --> next 1 loc stams x y z roll pitch yaw

SyntaxError: invalid syntax (1854906255.py, line 1)

In [ ]:
[el.values() for el in test_dataset.values()]

In [ ]:
# LSTM conv 1d

In [ ]:
from keras.layers.recurrent import LSTM

In [ ]:
from tensorflow.python.keras.layers.recurrent import LSTM, Bidirectional

In [ ]:
model = Sequential([
    LSTM(50, activation='tanh', return_sequences=True, input_shape=(10, 5)),  # First LSTM layer
    LSTM(30, activation='tanh'),  # Second LSTM layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
tf.keras.layers.Bidirectional(
    layer, merge_mode='concat'
)


In [ ]:
class Dataprepare():
    def __init__(dataset):
        self.dataset = dataset


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        current_target = self.target[idx]
        current_tweet = self.tweets[idx]
        sequence = []
        for word in current_tweet:
            if word in self.word_to_idx.keys():
                sequence.append(self.word_to_idx[word])

        return {
                'x': torch.tensor(sequence, dtype=torch.long),
                'y': torch.tensor(current_target, dtype=torch.long)
            }


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Dummy baseline

### read test dataset

In [ ]:
import numpy as np

NSECS_IN_SEC = 1000000000

def secs_to_nsecs(secs: float):
    return int(secs * NSECS_IN_SEC)

def nsecs_to_secs(nsecs: int):
    return float(nsecs) / NSECS_IN_SEC

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

In [ ]:
pip install keras==2.10.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
np.__version__

'2.2.3'

### simple pose prediction logic without taking into account control states

In [ ]:
def localization_df_to_poses(loc, contrl):
    loc_contrl = pd.concat([loc[74:1574].reset_index(),contrl.drop(columns='stamp_ns').reset_index()], axis=1).reset_index().drop(columns=['index','level_0'])
    poses = []
    for stamp_ns, x, y, yaw,acceleration_level, steering, in zip(loc_contrl['stamp_ns'], loc_contrl['x'], loc_contrl['y'], loc_contrl['yaw'], loc_contrl['acceleration_level'],loc_contrl['steering']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses


In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

In [ ]:
localization_df_to_poses(loc_df)


In [ ]:

import numpy as np

In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

# naive estimation of speed at last known localization pose
def dummy_estimate_last_speed(localization_poses):
    last_pose = localization_poses[-1]

    start_pose_idx = -1
    for i, pose in enumerate(localization_poses, start=1-len(localization_poses)):
        start_pose_idx = i
        if nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(pose['stamp_ns']) > 1.: # sec
            break

    start_pose = localization_poses[start_pose_idx]
    dt_sec = nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(start_pose['stamp_ns'])

    if dt_sec > 1e-5:
        return np.linalg.norm(last_pose['pos'][:2] - start_pose['pos'][:2]) / dt_sec
    return 5. # some default value

def dummpy_predict_pose(last_loc_pose: dict, last_speed: float, prediction_stamp: int):
    dt_sec = nsecs_to_secs(prediction_stamp) - nsecs_to_secs(last_loc_pose['stamp_ns'])
    distance = dt_sec * last_speed
    direction = yaw_direction(last_loc_pose['yaw'])
    pos_translate = direction * distance
    return {"pos": last_loc_pose['pos'] + pos_translate, 'yaw': last_loc_pose['yaw']}

In [ ]:
def predict_testcase(testcase: dict):
    loc_df = testcase['localization']
    localization_poses = localization_df_to_poses(loc_df)

    last_loc_pose = localization_poses[-1]
    last_speed = dummy_estimate_last_speed(localization_poses)

    predicted_poses = []
    for stamp in testcase['requested_stamps']['stamp_ns']:
        pose = dummpy_predict_pose(last_loc_pose, last_speed, stamp)
        predicted_poses.append(pose)

    predictions = {}
    predictions['stamp_ns'] = testcase['requested_stamps']['stamp_ns']
    predictions['x'] = [pose['pos'][0] for pose in predicted_poses]
    predictions['y'] = [pose['pos'][1] for pose in predicted_poses]
    predictions['yaw'] = [pose['yaw'] for pose in predicted_poses]
    return pd.DataFrame(predictions)

def predict_test_dataset(test_dataset: dict):
    predictions = {}
    for testcase_id, testcase in tqdm(test_dataset.items()):
        predictions[testcase_id] = predict_testcase(testcase)
    return predictions

### make prediction for requested stamps

In [ ]:
predictions = predict_test_dataset(train_dataset)
len(predictions)

  0%|          | 0/42000 [00:00<?, ?it/s]


KeyError: 'requested_stamps'

In [ ]:
test_predictions = predict_test_dataset(test_dataset)
len(test_predictions)

100%|██████████| 8000/8000 [00:20<00:00, 384.23it/s]


8000

In [ ]:
test_predictions

{0:         stamp_ns            x            y       yaw
 0     5000888836 -1490.905035 -1310.813635  2.047693
 1     5040043013 -1490.955001 -1310.716927  2.047693
 2     5079989560 -1491.005979 -1310.618261  2.047693
 3     5120797471 -1491.058057 -1310.517468  2.047693
 4     5165218288 -1491.114744 -1310.407751  2.047693
 ..           ...          ...          ...       ...
 370  19800297520 -1509.791352 -1274.259897  2.047693
 371  19840164576 -1509.842228 -1274.161428  2.047693
 372  19880314719 -1509.893466 -1274.062259  2.047693
 373  19920913971 -1509.945277 -1273.961981  2.047693
 374  19960655346 -1509.995993 -1273.863822  2.047693
 
 [375 rows x 4 columns],
 1:         stamp_ns           x           y      yaw
 0     5039398489 -920.724867 -106.410868  2.13907
 1     5079235758 -920.724874 -106.410856  2.13907
 2     5119629730 -920.724882 -106.410845  2.13907
 3     5159383587 -920.724889 -106.410833  2.13907
 4     5199602149 -920.724897 -106.410821  2.13907
 ..          

### write predictions

In [ ]:
def write_predictions(dataset_predictions: dict, prediction_file_path: str):
    prediction_list = []
    for testcase_id, prediction in tqdm(dataset_predictions.items()):
        prediction['testcase_id'] = [testcase_id] * len(prediction)
        prediction_list.append(prediction)
    predictions_df = pd.concat(prediction_list)
    predictions_df = predictions_df.reindex(columns=["testcase_id", "stamp_ns", "x", "y", "yaw"])
    print(len(predictions_df))
    predictions_df.to_csv(prediction_file_path, index=False, header=True)

In [ ]:
write_predictions(test_predictions, os.path.join(ROOT_DATA_FOLDER, "predictions.csv"))

# Calculate metric

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.


### implementation of the metric calculation

In [ ]:
import numpy as np
import pandas as pd

SEGMENT_LENGTH = 1.

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

def build_car_points(x_y_yaw):
    directions = np.vstack(yaw_direction(x_y_yaw[:, -1]))

    front_points = x_y_yaw[:, :-1] + SEGMENT_LENGTH * directions.T
    points = np.vstack([x_y_yaw[:, :-1], front_points])
    return points

def build_car_points_from_merged_df(df: pd.DataFrame):
    points_gt = df[['x_gt', 'y_gt', 'yaw_gt']].to_numpy()
    points_pred = df[['x_pred', 'y_pred', 'yaw_pred']].to_numpy()

    points_gt = build_car_points(points_gt)
    points_pred = build_car_points(points_pred)
    return points_gt, points_pred

def calculate_metric_testcase(df: pd.DataFrame):
    points_gt, points_pred = build_car_points_from_merged_df(df)

    metric = np.mean(np.sqrt(2. * np.mean((points_gt - points_pred) ** 2, axis=1)))
    return metric

def calculate_metric_dataset(ground_truth_df: pd.DataFrame, prediction_df: pd.DataFrame):
    assert (len(ground_truth_df) == len(prediction_df))

    df = ground_truth_df.merge(prediction_df, on=['testcase_id', 'stamp_ns'], suffixes=['_gt', '_pred'])

    metric = df.groupby('testcase_id').apply(calculate_metric_testcase)
    return np.mean(metric)